# CONFLUENCE Tutorial: Point-Scale Workflow (Flux Tower Example)

This notebook demonstrates CONFLUENCE's simplest spatial configuration: point-scale modeling. We'll simulate vertical processes at a single location, typical for flux tower sites, weather stations, or snow monitoring locations.

## Overview

Point-scale modeling focuses on vertical processes without spatial heterogeneity:
- Energy balance
- Soil moisture dynamics  
- Snow accumulation and melt
- Evapotranspiration

## Learning Objectives

1. Understand point-scale modeling in CONFLUENCE
2. Configure CONFLUENCE for single-point simulations
3. Compare with spatially distributed approaches
4. Analyze point-scale model outputs

## 1. Understanding Point-Scale Modeling

Before we simulate entire watersheds, we need to understand the vertical processes at a single point.

### When to Use Point-Scale Modeling:

- **Flux tower validation**: Compare model outputs with eddy covariance measurements
- **Process understanding**: Study vertical water and energy fluxes without spatial complexity
- **Parameter calibration**: Optimize parameters at well-instrumented sites
- **Model development**: Test new parameterizations before scaling up
- **SNOTEL/Weather stations**: Validate snow accumulation and melt processes

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import contextily as cx
import xarray as xr
import numpy as np

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

## 2. Create Point-Scale Configuration

For point-scale modeling, we need to modify our configuration to:
1. Set `SPATIAL_MODE: Point`
2. Create a minimal domain (single point with small buffer)
3. Focus on vertical processes

We'll use a SNOTEL site as our example: Loveland Pass in Colorado.

In [ ]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Verify paths exist
if not CONFLUENCE_CODE_DIR.exists():
    raise FileNotFoundError(f"CONFLUENCE code directory not found: {CONFLUENCE_CODE_DIR}")

if not CONFLUENCE_DATA_DIR.exists():
    print(f"Data directory doesn't exist. Creating: {CONFLUENCE_DATA_DIR}")
    CONFLUENCE_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Load template configuration
config_template_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_template.yaml'

# Read config file
with open(config_template_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update paths and settings for point-scale
config_dict['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config_dict['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)

# Modify for point-scale modeling
config_dict['DOMAIN_NAME'] = 'loveland_pass_snotel'
config_dict['EXPERIMENT_ID'] = 'point_scale_tutorial'
config_dict['SPATIAL_MODE'] = 'Point'  # Key setting for point-scale
config_dict['POUR_POINT_COORDS'] = '39.68/-105.88'  # Loveland Pass SNOTEL site

# Use lumped configuration (single unit)
config_dict['DOMAIN_DEFINITION_METHOD'] = 'lumped'
config_dict['DOMAIN_DISCRETIZATION'] = 'GRUs'

# Update experiment period for tutorial
config_dict['EXPERIMENT_TIME_START'] = '2020-10-01 00:00'
config_dict['EXPERIMENT_TIME_END'] = '2021-09-30 23:00'

# Set forcing data parameters
config_dict['FORCING_DATASET'] = 'ERA5'
config_dict['FORCING_VARIABLES'] = 'default'

# Save point-scale configuration to temporary file
temp_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_point_notebook.yaml'
with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f)

print("=== Point-Scale Configuration ===")
print(f"Domain Name: {config_dict['DOMAIN_NAME']}")
print(f"Spatial Mode: {config_dict['SPATIAL_MODE']}")
print(f"Location: {config_dict['POUR_POINT_COORDS']}")
print(f"Period: {config_dict['EXPERIMENT_TIME_START']} to {config_dict['EXPERIMENT_TIME_END']}")
print(f"Forcing Data: {config_dict['FORCING_DATASET']}")

## 3. Initialize CONFLUENCE for Point-Scale

In [ ]:
# Initialize CONFLUENCE with point-scale configuration
confluence = CONFLUENCE(temp_config_path)

print("=== CONFLUENCE Point-Scale Setup ===")
print(f"Project directory: {confluence.managers['project'].project_dir}")
print(f"Spatial mode: {confluence.config['SPATIAL_MODE']}")
print(f"Domain: {confluence.config['DOMAIN_NAME']}")

## 4. Setup Project Structure

In [ ]:
# Step 1: Project Initialization
print("=== Step 1: Project Initialization ===")
print("Creating point-scale project structure...")

# Setup project
project_dir = confluence.managers['project'].setup_project()

# Create pour point (in this case, our SNOTEL location)
pour_point_path = confluence.managers['project'].create_pour_point()

# List created directories
print("\nCreated directories:")
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        print(f"  📁 {item.name}")

print("\nNote: For point-scale modeling, we still use the same directory structure")
print("but the spatial extent is minimal (single point with small buffer).")

## 5. Geospatial Domain Definition - Data acquisition 

In [ ]:
# Acquire attributes
print("Acquiring geospatial attributes for point location...")
print(f"Minimal bounding box: {confluence.config.get('BOUNDING_BOX_COORDS')}")
confluence.managers['data'].acquire_attributes()

## 6. Geospatial Domain Definition - Domain creation

In [ ]:
# Geospatial Domain Definition and Analysis
print("=== Step 2: Geospatial Domain Definition and Analysis ===")

# Define domain
print("\nDefining minimal domain for point-scale simulation...")
watershed_path = confluence.managers['domain'].define_domain()
# Discretize domain (single HRU for point-scale)
print("\nCreating single HRU for point-scale simulation...")
hru_path = confluence.managers['domain'].discretize_domain()

# Check outputs
print("\nDomain definition complete:")
print(f"  - Watershed defined: {watershed_path is not None}")
print(f"  - HRUs created: {hru_path is not None}")

# Verify single HRU
hru_dir = project_dir / 'shapefiles' / 'catchment'
if hru_dir.exists():
    hru_files = list(hru_dir.glob('*.shp'))
    if hru_files:
        hru_gdf = gpd.read_file(hru_files[0])
        print(f"  - Number of HRUs: {len(hru_gdf)} (should be 1 for point-scale)")

## 7. Model Agnostic Data Pre-Processing - Data Acquisition

In [ ]:
# Step 3: Model Agnostic Data Pre-Processing
print("=== Step 3: Model Agnostic Data Pre-Processing ===")

# Process observed data (if we had real SNOTEL data, it would be processed here)
print("Processing observed data...")
# Note: We skip this for synthetic data, but normally would process SNOTEL data here

# Acquire forcings
print(f"\nAcquiring forcing data for point location...")
print(f"Dataset: {confluence.config['FORCING_DATASET']}")
confluence.managers['data'].acquire_forcings()

## 8. Model Agnostic Data Pre-Processing - Remapping and zonal statistics

In [ ]:
# Run model-agnostic preprocessing
print("\nRunning model-agnostic preprocessing...")
print("For point-scale simulations:")
print("  - No spatial averaging needed")
print("  - Apply elevation corrections if needed")
print("  - Convert units to model requirements")
confluence.managers['data'].run_model_agnostic_preprocessing()

print("\nModel-agnostic preprocessing complete")

## 9. Model Specific - Pre Processing 

In [ ]:
# Step 4: Model Specific Processing and Initialization
print("=== Step 4: Model Specific Processing and Initialization ===")

# Preprocess models
print(f"Preparing {confluence.config['HYDROLOGICAL_MODEL']} input files for point-scale simulation...")
print("Point-scale specific settings:")
print("  - Single HRU configuration")
print("  - No lateral flow between units")
print("  - Focus on vertical processes")
confluence.managers['model'].preprocess_models()

## 10. Model Specific - Initialisation

In [ ]:
# Run models
print(f"\nRunning {confluence.config['HYDROLOGICAL_MODEL']} for point-scale simulation...")
confluence.managers['model'].run_models()

print("\nPoint-scale model run complete")

## Summary: Point-Scale Modeling Insights

### What We Accomplished:

1. **Set up point-scale simulation** for a SNOTEL site
2. **Minimal spatial domain** with single HRU
3. **Focused on vertical processes** without lateral flow
4. **Analyzed key variables** relevant to point observations

### Key Differences from Spatial Models:

- **No spatial heterogeneity**: Single point representation
- **No routing**: All processes occur at one location
- **Direct comparison**: Easy validation with point observations
- **Computational efficiency**: Fast execution for testing

### Applications of Point-Scale Modeling:

1. **Parameter calibration**: Optimize parameters at well-observed sites
2. **Process validation**: Test model physics against observations
3. **Model development**: Develop new parameterizations
4. **Sensitivity analysis**: Understand parameter influence

### Next Steps:

1. **Scale up to lumped basin**: Use calibrated parameters
2. **Compare multiple sites**: Test model transferability
3. **Ensemble simulations**: Explore parameter uncertainty
4. **Process experiments**: Test different model configurations